In [1]:
# Analisis de datos
import pandas as pd
import numpy as np
import math

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from scipy import stats

# Fechas y Horas
import time
from datetime import date
from datetime import timedelta

# Visualizacion
import matplotlib.pyplot as plt
import seaborn as sns

# Propiedas y visualizacion
sns.set_theme(style='whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)

# Alertas
import warnings
warnings.filterwarnings("ignore")
    
# Configuraciones
pd.set_option('float_format', '{:,.2f}'.format) #Ajusta formato de números
pd.set_option('display.max_columns', None) #Muestra todas la columnas
pd.set_option('display.max_colwidth', None) #Mostrar el NUM_IDENT completo

### 0. Parámetros y Funciones de apoyo

In [2]:
# Funciones de Apoyo
# Función para mostrar las frecuencias absolutas y relativas de un campo
def volumetria(df,columna,orden):
    resultado = df.groupby([columna]).agg(Cantidad=(columna, 'count')).reset_index()
    total = resultado['Cantidad'].sum()
    resultado['% Total'] = (resultado['Cantidad'] / total) * 100
    if orden == 0:
        resultado = resultado.sort_values('% Total', ascending=False)
    elif orden == 1:
        resultado = resultado.sort_values(columna, ascending=True)
    else:
        return print('En orden solo puede escoger los valores 0,1')
    
    # Calcular el % Total Acumulado
    resultado['% Acumulado'] = resultado['% Total'].cumsum()
    return resultado

# Revision de Missings
def revision_missings(df):
    df_missings = pd.DataFrame({
        'COLUMNA': df.columns,
        'NRO_NULL': df.isna().sum(),
        '%_NULL': round(df.isna().mean() * 100, 2).astype(str) + '%'
    }).reset_index(drop=True)

    return df_missings

# Revision de Outliers General
def outliers_col(df):
    resultados = []
    for columna in df:
        if df[columna].dtype != object:
            q1 = stats.scoreatpercentile(df[columna].dropna(), 25)
            q3 = stats.scoreatpercentile(df[columna].dropna(), 75)
            iqr = q3 - q1
            lim_inf = q1 - 1.5 * iqr
            lim_sup = q3 + 1.5 * iqr
            n_outliers_inf = len(df[(df[columna] < lim_inf)])
            n_outliers_sup = len(df[(df[columna] > lim_sup)])
            resultados.append([df[columna].name, n_outliers_inf, n_outliers_sup])
    
    df_resultados = pd.DataFrame(resultados, columns=['NOMBRE_COLUMNA', 'NRO_OUTLIERS_INF', 'NRO_OUTLIERS_SUP'])
    return df_resultados

# Revision de Outliers Individual
def rev_outliers(df,campo):
    cantidad = df[campo].count()
    nulls = df[campo].isna().sum()
    unicos = df[campo].nunique()
    porc_nulls = round(nulls/ df.shape[0],4)*100
    print(f'Nro Filas: {cantidad:,}.')
    print(f'Nro Nulls: {nulls:,}. Un {porc_nulls}% son nulls.')
    print(f'Nro Valores Unicos: {unicos:,}.')

    print(f'\nDeciles de {campo}:')
    print(df[campo].quantile([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 0.99]))

    # Calcular los límites utilizando el rango intercuartílico (IQR)
    q1 = df[campo].quantile(0.25)
    q3 = df[campo].quantile(0.75)
    iqr = q3 - q1
    lower_limit = q1 - 1.5 * iqr
    upper_limit = q3 + 1.5 * iqr

    print(f'\nLímite superior: {round(upper_limit,2)} y Límite inferior: {round(lower_limit,2)}')
    print(f'Valor mínimo: {df[campo].min()}')
    print(f'Valor máximo: {df[campo].max()}')

    # Identificar los valores atípicos
    outliers = df[(df[campo] < lower_limit) | (df[campo] > upper_limit)]
    # Calcular la cantidad de outliers
    print(f'\nCantidad de outliers: {outliers.shape[0]} de {df.shape[0]}. Es un {round(outliers.shape[0]/df.shape[0],2)*100}%.')

    # Graficar el boxplot
    sns.boxplot(x=df[campo])
    # Agregar título al gráfico
    plt.title('Boxplot de '+campo)
    # Mostrar el gráfico
    plt.show()

def visualizar_kde(df, excluir_columnas, num_columnas=3, figsize=(15, 5)):
    # Filtrar las columnas que no están en la lista de exclusión
    columnas = df.drop(columns=excluir_columnas).columns

    # Definir el número de filas basado en las columnas
    num_filas = math.ceil(len(columnas) / num_columnas)

    # Crear la figura y los ejes
    fig, axes = plt.subplots(num_filas, num_columnas, figsize=(figsize[0], num_filas * figsize[1]))
    axes = axes.flatten()  # Aplanar la matriz de ejes para facilitar el acceso

    # Iterar sobre cada columna y graficar en cada subgráfico
    for i, col in enumerate(columnas):
        sns.histplot(df[col], kde=True, ax=axes[i])
        axes[i].set_title(f'Distribución de {col}')

    # Eliminar cualquier gráfico vacío si el número de columnas no es múltiplo exacto de filas * columnas
    for i in range(len(columnas), len(axes)):
        fig.delaxes(axes[i])

    # Ajustar el espacio entre los gráficos
    plt.tight_layout()
    plt.show()

# Función para limitar por la izquierda
def limitar_por_izquierda(df, columna, limite_inferior):
    df[columna][df[columna] < df[columna].quantile(limite_inferior)] = df[columna].quantile(limite_inferior)
    return df

# Función para limitar por la derecha
def limitar_por_derecha(df, columna, limite_superior):
    df[columna][df[columna] > df[columna].quantile(limite_superior)] = df[columna].quantile(limite_superior)
    return df

### 1. Importacion Data

In [3]:
import unidecode

directory_contactos = "C:/Users/BW439WF/Downloads/Datathon DMC/Datathon_DMC/data/02_raw/"
df_compromisos = pd.read_excel(directory_contactos + 'BBDD_COMPROMISOS.xlsx')

# Función para limpiar nombres de columnas
def clean_column_names(columns):
    return [unidecode.unidecode(col.upper().replace(" ", "_").replace(":", "")) for col in columns]

# Aplicar la función a las columnas del DataFrame
df_compromisos.columns = clean_column_names(df_compromisos.columns)

# Resumen Data
#print(f'Nro Clientes: {df_compromisos["COD_CLIENTE"].nunique():,}')
print(f'Nro Filas: {df_compromisos.shape[0]:,}')
df_compromisos.head()

Nro Filas: 192,409


,CONTACTO_ID_COMPLETO,ID_COMPLETO,CONTACTO_NOMBRE,IMPORTE_DONACION,TIPO_DE_COMPROMISO,COMPROMISO_DE_DONACION_TIPO_DE_REGISTRO,CANTIDAD_DE_DONACIONES_PLANIFICADAS,NUMERO_DE_CUOTAS,FRECUENCIA,MEDIO_DE_PAGO,ENTIDAD_COBRADORA,TIPO_DE_CUENTA,TIPO_DE_TARJETA,BANCO_EMISOR,CANAL_DE_LA_CAMPANA,CODIGO_DE_AUDITORIA,CAMPANA/ACTIVIDAD,CAMPANA_DE_FIDELIZACION,CONTACTO_FECHA_DE_CAPTACION,FECHA_DEL_COMPROMISO_DE_DONACION,FECHA_ESTABLECIDA,CODIGO_DE_UBICACION,PUNTO_DE_CAPTACION,ESTADO_DEL_COMPROMISO,CONTACTO_DONANTE_ACTIVO,COMPROMISO_ACTIVO,FECHA_DE_BAJA,TIPO_DE_BAJA,MOTIVO_DE_BAJA,CONTACTO_FECHA_DE_PRIMER_PAGO,CONTACTO_FECHA_DE_ULTIMA_DONACION,CONTACTO_CANTIDAD_CUOTAS_PAGADAS_GLOBAL,CONTACTO_FECHA_DE_PRIMERA_DONACION,CONTACTO_TOTAL_DONADO
0,0031R00001uhz7aQAA,a091R00000LyIpHQAV,Karen Hellen,20.00,Primer Compromiso,DI - Regular Tiempo determinado,11.00,12.00,Mensual,Tarjeta de Crédito,NaN,NaN,Master Card,NaN,F2F,F2F_P4,RESERVA DEL SGIP P4,NaN,2018-10-15,2018-10-22,2018-11-01,Lima,METRO ATOCONGO,Cerrado,0,0,2019-11-07,Baja voluntaria,Problemas económicos,2018-11-01,NaT,11,NaT,0.00
1,0031R00001uhz7bQAA,a091R00000LyIpIQAV,Alberto Jesus,60.00,Primer Compromiso,PI - Padrinazgo individual,27.00,NaN,Mensual,Tarjeta de Crédito,NaN,NaN,Visa,BBVA CONTINENTAL,F2F,NaN,F2F IFFC,NaN,2018-10-15,2018-10-22,2018-11-01,Lima,WONG CHACARILLA,Cerrado,0,0,2020-08-13,Baja voluntaria,Problemas económicos,2018-11-01,NaT,22,NaT,0.00
2,0031R00001uhz7cQAA,a091R00000abp74QAA,Paolo Carlos,60.00,Nuevo por cambio de datos clave,PI - Padrinazgo individual,43.00,NaN,Mensual,Tarjeta de Crédito,NaN,NaN,Visa,BANCO DE CRÉDITO DEL PERÚ,F2F,NaN,FACE TO FACE,NaN,2018-10-16,2019-11-26,2019-12-26,Lima,ATENCION AL DONANTE - F2F,Abierto,0,0,NaT,NaN,NaN,2018-11-01,NaT,12,NaT,0.00
3,0031R00001uhz7cQAA,a091R00000LyIpJQAV,Paolo Carlos,60.00,Primer Compromiso,PI - Padrinazgo individual,18.00,NaN,Mensual,Tarjeta de Crédito,NaN,NaN,Visa,BANCO DE CRÉDITO DEL PERÚ,F2F,NaN,FACE TO FACE,NaN,2018-10-16,2018-10-22,2018-11-01,Lima,RIPLEY PRIMAVERA,Cerrado,0,0,2019-11-26,Cierre de compromiso,Cambio de Datos,2018-11-01,NaT,12,NaT,0.00
4,0031R00001uhz7dQAA,a091R00000LyIpKQAV,Jose Carlos,10.00,Primer Compromiso,DI - Regular,7.00,NaN,Mensual,Débito Automático,Banco de Crédito del Perú,Ahorros,NaN,NaN,F2F,NaN,F2F IFFC,NaN,2018-10-17,2018-10-22,2018-11-01,Lima,RIPLEY PLAZA SAN MIGUEL,Cerrado,0,0,2018-12-06,Baja voluntaria,Problemas económicos,2018-11-01,NaT,2,NaT,0.00


In [5]:
df_compromisos['CONTACTO_ID_COMPLETO'].value_counts()

CONTACTO_ID_COMPLETO
0033600001J6eS9AAJ    24
0031R000025t7kYQAQ    20
0033600001J6fmOAAR    16
0031R00002N1afqQAB    15
0033600001AE741AAD    15
                      ..
0033600001AE3goAAD     1
0033600001AE3gnAAD     1
0033600001AE3gMAAT     1
0033600001AE3gmAAD     1
003UX00000I33INYAZ     1
Name: count, Length: 145081, dtype: int64

In [4]:
import unidecode  # Asegura que tienes instalada esta librería con: pip install unidecode
directory_contactos = "C:/Users/BW439WF/Downloads/Datathon DMC/Datathon_DMC/data/02_raw/"
df_contactos = pd.read_excel(directory_contactos + 'BBDD_CONTACTOS.xlsx')

# Función para limpiar nombres de columnas
def clean_column_names(columns):
    return [unidecode.unidecode(col.upper().replace(" ", "_")) for col in columns]

# Aplicar la función a las columnas del DataFrame
df_contactos.columns = clean_column_names(df_contactos.columns)

# Resumen Data
print(f'Nro Clientes: {df_contactos["ID_COMPLETO"].nunique():,}')
print(f'Nro Filas: {df_contactos.shape[0]:,}')
df_contactos.head()

Nro Clientes: 147,513
Nro Filas: 147,513


,ID_COMPLETO,PAIS_DE_CORREO,ESTADO_O_PROVINCIA_DE_CORREO,CODIGO_POSTAL_DE_CORREO,CIUDAD_DE_CORREO,NOMBRE,GENERO,TRATAMIENTO,ESTADO_CIVIL,TIENE_HIJOS,HOBBIE,TIENE_NIETOS,NIVEL_DE_ESTUDIOS,TRABAJA_ACTUALMENTE,OCUPACION
0,0031R00001uhz7IQAQ,Perú,Lima,Lima,San Borja,Gledy Georgina,Femenino,Srita.,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0031R0000281tkvQAA,Perú,Lima,Lima,San Borja,Fredy Enrique,Masculino,Sr.,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0031R00002KqNYbQAN,Perú,Lima,Lima,Santiago De Surco,Alvaro Patricio,Masculino,Sr.,Soltero/a?,No,NaN,NaN,NaN,NaN,NaN
3,0031R00002hJzb5QAC,Perú,Lambayeque,Chiclayo,Chiclayo,Anton Cesar,Masculino,Sr.,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0033600001ADzauAAD,Perú,Lima,Lima,Comas,Diana Alison Milagros,Femenino,Sra.,NaN,NaN,NaN,NaN,NaN,NaN,NaN
